In [1]:
#調査ターゲットとなる都道府県のURLをリスト化
import urllib
url_list=[]

p_list=["ibaraki","saitama","tokyo","kanagawa","chiba","tochigi","gumna","yamanashi",
     "osaka","hyogo","kyoto","shiga","nara","wakayama",
     "aichi","shizuoka","gifu","mie","nagano",
     "hokkaido","aomori","iwate","miyagi","akita","yamagata","fukushima",
     "niigata","toyama","ishikawa","fukui",
     "tottori","shimane","okayama","hiroshima","yamaguchi",
     "tokushima","kagawa","ehime","kochi",
     "fukuoka","saga","nagasaki","kumamoto","oita","miyazaki","kagoshima","okinawa"]
#ターゲットとなる都道府県（prefecture）のリスト

for alfa in p_list:
    t_url= "https://timee.co.jp/jobs/{}?category=sales&page=".format(alfa)
    url_list.append(t_url)

print(url_list)

['https://timee.co.jp/jobs/ibaraki?category=sales&page=', 'https://timee.co.jp/jobs/saitama?category=sales&page=', 'https://timee.co.jp/jobs/tokyo?category=sales&page=', 'https://timee.co.jp/jobs/kanagawa?category=sales&page=', 'https://timee.co.jp/jobs/chiba?category=sales&page=', 'https://timee.co.jp/jobs/tochigi?category=sales&page=', 'https://timee.co.jp/jobs/gumna?category=sales&page=', 'https://timee.co.jp/jobs/yamanashi?category=sales&page=', 'https://timee.co.jp/jobs/osaka?category=sales&page=', 'https://timee.co.jp/jobs/hyogo?category=sales&page=', 'https://timee.co.jp/jobs/kyoto?category=sales&page=', 'https://timee.co.jp/jobs/shiga?category=sales&page=', 'https://timee.co.jp/jobs/nara?category=sales&page=', 'https://timee.co.jp/jobs/wakayama?category=sales&page=', 'https://timee.co.jp/jobs/aichi?category=sales&page=', 'https://timee.co.jp/jobs/shizuoka?category=sales&page=', 'https://timee.co.jp/jobs/gifu?category=sales&page=', 'https://timee.co.jp/jobs/mie?category=sales&pa

In [3]:
#３県テスト。うまく行った！
import csv
import time
import urllib.request as req
import os.path,random
import pprint
import requests
from bs4 import BeautifulSoup
import json


timee_e_list=[]
timee_list=[]

p_list=["osaka","tokushima","saga"]
#東京から東　p_list=["hokkaido","aomori","iwate","miyagi","akita","yamagata","fukushima","ibaraki","tochigi","gunma","saitama"
                #,"chiba"]
#東京のみ　p_list=["tokyo"]
#東京から西　p_list=["kanagawa","niigata","toyama","ishikawa","fukui","yamanashi","nagano","gifu","shizuoka","aichi","mie"
                #,"shiga","kyoto","osaka","hyogo","nara","wakayama","tottori","shimane","okayama","hiroshima","yamaguchi"
                #,"tokushima","kagawa","ehime","kochi","fukuoka","saga","nagasaki","kumamoto","oita","miyazaki","kagoshima"
                #,"okinawa"]
            
y=0
for y in range(len(p_list)):
    limit_list=[]
    
    num=1

    limit=1
    #このlimit=1は下のwhileが始まらないが故のダミー。while以下で正しい数字に置き換わり、全体への影響はない。
    while limit>0:
        url_2=requests.get("https://timee.co.jp/jobs/"+p_list[y]+"?category=sales&page={}".format(num)).text
        soup= BeautifulSoup(url_2,"html.parser")

        j_data= soup.find("script", {"type": "application/json"})
        data=j_data.text

        json_dict = json.loads(data)

        limit=json_dict['props']['pageProps']['pagination']['limit']
        limit_list.append(limit)
        time.sleep(2)
        num+=1

    print(limit_list)

    num=1
    kilo=0
    for x in range(len(limit_list)-1):
        target_url_1 = "https://timee.co.jp/jobs/"+p_list[y]+"?category=sales&page={}".format(num)
        print(target_url_1)
        count=limit_list[kilo]

        r = requests.get(target_url_1)
        soup_1 = BeautifulSoup(r.content, "lxml")

        json_part_1 = soup_1.find_all("script", {"type": "application/json"})

        for i in json_part_1:
            json_text_1 = i.get_text()

            a = json.loads(json_text_1)

            k=0
            for k in range(count):
                encryptId=a['props']['pageProps']['offers'][k]['encryptId']
                timee_e_list.append(encryptId)
                k +=1
                time.sleep(2)

        num +=1
        kilo +=1

        time.sleep(2)

print(timee_e_list)
#ここまでで欲しいサブページの暗唱番号を、timee_e_listの中に全て抽出

alfa_list=[]
blavo_list=[]
delta_list=[]

charlie=len(timee_e_list)

foxtrot=0 
for i in range(charlie):
    e=timee_e_list[foxtrot]
    target_url_2="https://timee.co.jp/jobs/offer/{}".format(e)

    r_2= requests.get(target_url_2) 
    soup_2= BeautifulSoup(r_2.content, "lxml")
        
    alfa=soup_2.find_all("p",class_="style_clientName__3mE6o")
    alfa_t= [x.text for x in alfa]
    alfa_list.append(alfa_t)
    blavo=soup_2.find_all("span",class_="style_date__2JtLJ")
    blavo_t= [x.text for x in blavo]
    blavo_list.append(blavo_t)
    delta=soup_2.find_all("span",class_="style_label__2iKq7")
    delta_t= [x.text for x in delta]
    delta_list.append(delta_t)

    time.sleep(2)
    foxtrot+=1

import pandas as pd

dfm = pd.DataFrame()
for i in range(len(blavo_list)):
    df = pd.DataFrame([[x, y] for x, y in zip(blavo_list[i], delta_list[i])])
    df.insert(0, '', alfa_list[i][0])
    dfm = pd.concat([dfm, df])

dfm.columns = ['店舗名', '募集日', '状況']
dfm.to_csv('timee_３県テスト20210105.csv', encoding="utf-8", index=False)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)